In [28]:
import requests
from lxml import html
import pandas as pd
from datetime import date
import re

In [4]:
words_to_look = [
    'salud',
    'farmacoeconomía',
    'medicamentos',
    'health',
    'pharmacoeconomics',
    'medicines',
    'santé ',
    'pharmacoéconomie',
    'médicaments',
    'saude',
    'farmacoeconomia',
    'medicamentos'
]

today = date.today().strftime("%d/%m/%Y")
source = 'OECD'
file_name = './oecd_ops.csv'
main_url = 'https://www.oecd.org/callsfortenders/listofallcallsfortenders.htm'

### Generacion de csv

In [2]:
df = pd.DataFrame(columns=['url_detail_id','scrapped_day','title','opening_date',
                           'closing_date','source','is_alert','location'])

In [5]:
df.to_csv(file_name, index=False, encoding='utf-8', header=True)

In [6]:
del df

### probando

In [7]:
#lee la tabla de la pagina
df = pd.read_csv(file_name, encoding='utf-8')

In [8]:
df

,url_detail_id,scrapped_day,title,opening_date,closing_date,source,is_alert,location


In [9]:
def get_page(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    return tree

In [49]:
def get_by_xpath_and_clean(tree, xpath, i=0):
    """"""
    try:
        if i=='join':
            text = tree.xpath(xpath)
            text = re.sub(r'[\n\t\xa0]', '', ' '.join(text)).strip()
            return text
        else:
            text = tree.xpath(xpath)[i]
            text = re.sub(r'[\n\t\xa0]', '', text).strip()
            return text
    except:
        return None

In [10]:

response = get_page(main_url)

In [33]:
#get a list of jobs and ommits the first one and the last one
response.xpath('//tbody/tr')[1:-1]

[<Element tr at 0x1e52268be00>,
 <Element tr at 0x1e5232dd590>,
 <Element tr at 0x1e522726090>,
 <Element tr at 0x1e522726680>]

In [34]:
jobs = response.xpath('//tbody/tr')[1:-1]

In [44]:
get_by_xpath_and_clean(jobs[0], './td[2]/text()')

'RFQ_646'

In [46]:
'https://www.oecd.org' + get_by_xpath_and_clean(jobs[0], '(./td[4]/p//a)[1]/@href')

'https://www.oecd.org/callsfortenders/RFQ_646_WEB.pdf'

In [53]:
get_by_xpath_and_clean(jobs[0], './td[4]/descendant::text()', i='join')

'Maintenance of the canopy glass ceiling Contact: kevin.sejean@oecd.org'

In [55]:
get_by_xpath_and_clean(jobs[0], './td[5]/text()')

'06.03.2023'

In [56]:
get_by_xpath_and_clean(jobs[0], './td[6]/text()', i='join')

'30.03.2023 2.00 pm (Paris time)'

In [57]:
for i, job in enumerate(jobs):
    #get the url of detail
    detail_url = 'https://www.oecd.org' + get_by_xpath_and_clean(job, '(./td[4]/p//a)[1]/@href')
    
    #looks if the detail url is already in the dataset
    if df['url_detail_id'][df['url_detail_id']==detail_url].any():
        print('this job is already in the dataset')
        continue
    #if not  exist, get the detail
    else:
        print('nueva oportunidad encontrada')
        #type RFQ
        type_RFQ = get_by_xpath_and_clean(job, './td[2]/text()')
        #get the title
        title = get_by_xpath_and_clean(job, './td[4]/descendant::text()', i='join')
        #get the location (type RFQ)
        location = get_by_xpath_and_clean(job, './td[2]/text()')
        #Opening Date:
        opening_date = get_by_xpath_and_clean(job, './td[5]/text()')
        #Closing Date
        closing_date = get_by_xpath_and_clean(job, './td[6]/text()', i='join')
        
        #find the body of the job and look for the words_to_look to appear once at least
        is_alert = False

        text_for_alert = title.strip().lower()
        if any(word in text_for_alert for word in words_to_look):
            is_alert = True
        #add the new job to the dataset
        df = df.append({'url_detail_id': detail_url, 'scrapped_day': today,  'title': title, 
                'opening_date': opening_date, 'closing_date': closing_date,'location': location,
                'is_alert': is_alert, 'source': source}, ignore_index=True)

nueva oportunidad encontrada
nueva oportunidad encontrada
nueva oportunidad encontrada
nueva oportunidad encontrada


In [58]:
df

,url_detail_id,scrapped_day,title,opening_date,closing_date,source,is_alert,location
0,https://www.oecd.org/callsfortenders/RFQ_646_W...,14/03/2023,Maintenance of the canopy glass ceiling Contac...,06.03.2023,30.03.2023 2.00 pm (Paris time),OECD,False,RFQ_646
1,https://www.oecd.org/callsfortenders/RFQ_645_W...,14/03/2023,Fire Safety Systems Coordination Contact: kevi...,06.03.2023,30.03.2023 2.00 pm (Paris time),OECD,False,RFQ_645
2,https://www.oecd.org/callsfortenders/RFQ_644_W...,14/03/2023,Audio Visual services Contact: kevin.sejean@o...,06.03.2023,30.03.2023 2.00 pm (Paris time),OECD,False,RFQ_644
3,https://www.oecd.org/callsfortenders/RFI.pdf,14/03/2023,Request for Information on the Common Transmis...,06.03.2023,30.03.2023 2.00 pm (Paris time),OECD,False,


In [27]:
#Type RFQ
response.xpath('//tbody/tr[2]/td[1]/text()')

['Call for Tenders']

In [35]:
#rfq number
response.xpath('//tbody/tr[2]/td[2]/text()')

['RFQ_646']

In [42]:
#detail_url //tbody/tr/td/p[1]//a[1]
response.xpath('(//tbody/tr[2]/td[4]/p//a)[1]/@href')

['/callsfortenders/RFQ_646_WEB.pdf']

In [26]:
#offer
response.xpath('//tbody/tr[2]/td[4]/descendant::text()')

['\n',
 'Maintenance of the canopy glass ceiling',
 'Contact:\xa0',
 'kevin.sejean@oecd.org',
 '\n']

In [54]:
#Launched on 
response.xpath('//tbody/tr[2]/td[5]/text()')

['06.03.2023\xa0']

In [24]:
#Deadline
response.xpath('//tbody/tr[2]/td[6]/text()')

['30.03.2023', '2.00 pm (Paris time)\xa0\xa0']